In [ ]:
import torch
import torch.nn as nn
from torchvision.transforms import transforms
# SINCE THE MNIST DATASET IS IN 28X28, PADDING OF 2 IS PUT ON C1 LAYER
class LeNet5_Dict_Out(nn.Module):
    def __init__(self):
        super(LeNet5_Dict_Out,self).__init__()
        self.quant = torch.quantization.QuantStub()
        self.dequant = torch.quantization.DeQuantStub()
        self.c1 = nn.Conv2d(in_channels=1,out_channels=6,kernel_size=5,stride=1,padding=2)
        self.c2 = nn.Conv2d(in_channels=6,out_channels=16,kernel_size=5,stride=1,padding=0)
        self.c3 = nn.Conv2d(in_channels=16,out_channels=120,kernel_size=5,stride=1,padding=0)
        # self.tanh = nn.Tanh()
        self.relu = nn.ReLU()
        self.max_pool = nn.MaxPool2d(kernel_size=2,stride=2)
        # self.fc0 = nn.Linear(in_features=400,out_features=120)
        self.fc1 = nn.Linear(in_features=120,out_features=84)
        self.fc2 = nn.Linear(in_features=84,out_features=10)

    def forward(self, img):
        out_dict = {}
        x = self.quant(img)
        # in 1x32x32 out 6x28x28
        x = self.c1(x)
        out_dict['c1'] = x
        # in 6x28x28 out 6x14x14
        # replace tanh with relu
        x = self.max_pool(x)
        out_dict['pool_c1'] = x
        x = self.relu(x)
        out_dict['relu_c1'] = x
        # in 6x14x14 out 16x10x10
        x = self.c2(x)
        out_dict['c2'] = x
        # in 16x10x10 out 16x5x5
        # replace tanh with relu
        x = self.relu(self.max_pool(x))
        out_dict['pool_c2'] = x
        # # in 16x5x5 out 400x1x1
        # x = torch.flatten(x,1)
        # x = self.tanh(self.fc0(x))
        # in 16x5x5 out 120x1x1
        # replace tanh with relu
        x = self.relu(self.c3(x))
        x = torch.flatten(x,1)
        out_dict['c3'] = x.int_repr()
        # replace tanh with relu
        x= self.fc1(x)
        out_dict['fc1'] = x
        x = self.relu(x)
        out_dict['fc1_relu'] = x
        x = self.fc2(x)
        out_dict['fc2'] = x
        x = self.dequant(x)
        return x, out_dict
    


In [3]:
from pathlib import Path
MODEL_NAME = "quantized_lenet5_mnist_20250703_1003.pth"
MODEL_PATH = Path("weights")
MODEL_PATH.mkdir(parents=True, exist_ok=True)
MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME


In [ ]:
import torchvision
from torch.utils.data import DataLoader
# device-agnostic setup
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# load test dataset
BATCH_SIZE = 32
test_dataset  = torchvision.datasets.MNIST(root='./data', train=False, download=False, transform=transforms.ToTensor())
imgs = torch.stack([img for img, _ in test_dataset], dim=0)
mean = imgs.view(1, -1).mean(dim=1)    # or imgs.mean()
std = imgs.view(1, -1).std(dim=1)     # or imgs.std()
print(f"MNIST mean: {mean.item():.6f}")
print(f"MNIST std:  {std.item():.6f}")
# create Transformation (converting from Image class to Tensor and normalize)
mnist_transforms = transforms.Compose([transforms.ToTensor(),])
                                    #    transforms.Normalize(mean=mean, std=std)])

test_dataset  = torchvision.datasets.MNIST(root='./data', train=False, download=False, transform=mnist_transforms)
class_names = test_dataset.classes

test_dataloader = DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=True)


# Step 1: Re-create model and set QAT config
model_fp32 = LeNet5_Dict_Out()
model_fp32.qconfig = torch.quantization.get_default_qat_qconfig('fbgemm')
torch.quantization.prepare_qat(model_fp32 , inplace=True)

# Step 2: Convert to quantized version
net = torch.quantization.convert(model_fp32 .eval(), inplace=False)
# Step 3: Load quantized weights
net.load_state_dict(torch.load(MODEL_SAVE_PATH))


test_loss, test_acc = 0, 0

net.to(device)

net.eval()

100.0%
100.0%
100.0%
100.0%


MNIST mean: 0.132515
MNIST std:  0.310480


c:\Users\user\miniconda3\envs\ml\lib\site-packages\torch\ao\quantization\observer.py:244: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  warnings.warn(
c:\Users\user\miniconda3\envs\ml\lib\site-packages\torch\ao\quantization\utils.py:408: UserWarning: must run observer before calling calculate_qparams. Returning default values.
  warnings.warn(
c:\Users\user\miniconda3\envs\ml\lib\site-packages\torch\_utils.py:425: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  device=storage.device,


LeNet5_Dict_Out(
  (quant): Quantize(scale=tensor([0.0256]), zero_point=tensor([17]), dtype=torch.quint8)
  (dequant): DeQuantize()
  (c1): QuantizedConv2d(1, 6, kernel_size=(5, 5), stride=(1, 1), scale=0.13094060122966766, zero_point=68, padding=(2, 2))
  (c2): QuantizedConv2d(6, 16, kernel_size=(5, 5), stride=(1, 1), scale=0.33193832635879517, zero_point=83)
  (c3): QuantizedConv2d(16, 120, kernel_size=(5, 5), stride=(1, 1), scale=0.5925276279449463, zero_point=74)
  (relu): ReLU()
  (max_pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): QuantizedLinear(in_features=120, out_features=84, scale=0.5568221807479858, zero_point=67, qscheme=torch.per_tensor_affine)
  (fc2): QuantizedLinear(in_features=84, out_features=10, scale=1.0586904287338257, zero_point=84, qscheme=torch.per_tensor_affine)
)

In [20]:
import numpy as np

def to_c_initializer(a: np.ndarray,
                     c_type: str = "uint8_t",
                     var_name: str = "tensor") -> str:
    """
    Convert a NumPy array of arbitrary rank to a C-style initializer.

    Parameters
    ----------
    a        : np.ndarray  – array to export
    c_type   : str         – C element type (e.g. uint8_t, int32_t, float)
    var_name : str         – variable name in the generated snippet

    Returns
    -------
    str – ready-to-paste C code:
          c_type var_name[dim0][dim1]...[dimN] = { ... };
    """
    # 1. Build the full C type with sizes
    dims = "".join(f"[{d}]" for d in a.shape)
    header = f"{c_type} {var_name}{dims} = "

    # 2. Recursively format with braces
    def rec_fmt(x):
        if x.ndim == 1:
            return "{" + ", ".join(map(str, x.tolist())) + "}"
        return "{" + ",\n ".join(rec_fmt(y) for y in x) + "}"

    body = rec_fmt(a) + ";"
    return header + body


# -------------------------------  demo  ---------------------------------
if __name__ == "__main__":
    img = np.arange(1*2*3*4, dtype=np.uint8).reshape(1, 2, 3, 4)
    code = to_c_initializer(img, "uint8_t", "image01")
    print(code)


uint8_t image01[1][2][3][4] = {{{{0, 1, 2, 3},
 {4, 5, 6, 7},
 {8, 9, 10, 11}},
 {{12, 13, 14, 15},
 {16, 17, 18, 19},
 {20, 21, 22, 23}}}};


uint8_t image01[1][2][3][4] = {{{{0, 1, 2, 3},
 {4, 5, 6, 7},
 {8, 9, 10, 11}},
 {{12, 13, 14, 15},
 {16, 17, 18, 19},
 {20, 21, 22, 23}}}};


In [7]:
from matplotlib import pyplot as plt
from PIL import Image
IMAGE_NAME = "img_01.png"
IMAGE_PATH = Path(Path.cwd() / "results" / "images" / IMAGE_NAME)

weights = torch.load(MODEL_SAVE_PATH)
img = Image.open(IMAGE_PATH).convert('L')  # 'L' for grayscale
# plt.imshow(img, cmap='gray')
img_tensor = mnist_transforms(img)
# print(img_tensor*255)
torch.set_printoptions(profile="full")
print(net.quant(img_tensor).int_repr())
img_tensor.unsqueeze_(0)  # shape: [1, 1, 28, 28]
output, out_dict = net(img_tensor)
# print(output)
# print(out_dict['c1'])

tensor([[[17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17,
          17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17],
         [17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17,
          17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17],
         [17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17,
          17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17],
         [17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17,
          17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17],
         [17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17,
          17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17],
         [17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17,
          17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17],
         [17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17,
          17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17],
         [17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17,

In [37]:
c1_real_scale = (weights['quant.scale'] * weights['c1.weight'].q_scale()) / weights['c1.scale']
print(c1_real_scale.item())
print(weights['c1.scale'])
print(weights['c2.weight'].q_scale())
print(weights['c2.scale'])
c2_real_scale = (weights['c1.scale'] * weights['c2.weight'].q_scale()) / weights['c2.scale']
print(c2_real_scale.item())
# print(weights['c2.zero_point'])
# print(weights['c2.weight'].q_zero_point())

0.0007882966310717165
tensor(0.1309)
0.005365171004086733
tensor(0.3319)
0.002116413554176688


In [25]:
c2_weight = to_c_initializer(weights['c2.weight'].int_repr().numpy(), "int8_t", "image01")
print(c2_weight)


int8_t image01[16][6][5][5] = {{{{-21, -3, -5, -18, -13},
 {27, 49, 2, -6, 0},
 {40, -12, -11, -28, -8},
 {5, 39, 13, -67, -35},
 {47, -3, -41, -10, -5}},
 {{2, -7, -12, -14, -20},
 {18, 4, -54, -7, 11},
 {20, -38, -71, 21, 24},
 {-7, -74, -32, 50, 45},
 {-56, -71, 11, 26, 42}},
 {{-36, -24, -25, -89, -32},
 {11, 21, -10, -37, 13},
 {39, 13, -44, -22, 25},
 {55, -53, -58, 16, 34},
 {-65, -17, -17, -28, -35}},
 {{31, -11, 8, -12, -39},
 {-11, -15, 8, -29, -36},
 {-64, -27, 43, 11, -4},
 {-14, 18, 24, 8, -41},
 {21, -1, 27, 7, -11}},
 {{20, 19, 10, 0, 11},
 {33, 19, -17, 10, -25},
 {1, 7, 24, 26, 10},
 {-44, 2, 23, 15, -51},
 {0, 9, 28, 28, -59}},
 {{-12, 13, -3, -4, 16},
 {39, 10, -12, 5, 2},
 {29, -44, -42, -25, -3},
 {-25, -62, -64, -16, -9},
 {-26, -4, 0, 32, 3}}},
 {{{-8, 8, -14, -27, -27},
 {5, 21, -15, 7, -35},
 {19, -34, 22, 39, 47},
 {12, -29, 19, 54, 30},
 {17, 27, 0, -6, 26}},
 {{-6, 12, 20, 34, 47},
 {-10, 23, -4, 4, 20},
 {-10, 4, 6, -18, -42},
 {4, 25, -8, -60, -42},
 {-4, 

## CHECK FOR C1 DIFFERENCE

In [22]:
c1_pytorch = out_dict['c1']
print(c1_pytorch.int_repr().to(torch.int8) - weights['c1.zero_point'])

# c1_manual = torch.load('img_01.png_c1.pt')
# torch.set_printoptions(profile="full")
# print(c1_pytorch[0,0,0:10,0:10].int_repr())
# print(torch.sum(c1_manual - c1_pytorch.int_repr())) 

tensor([[[[ -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,
            -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1],
          [ -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,
            -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1],
          [ -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,
            -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1],
          [ -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,
            -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1],
          [ -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,
            -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1],
          [ -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,
            -1,  -1,  -1,  -1,  -1,  -2,  -3,  -3,  -2,  -2,   0,   1,   0,  -1],
          [ -1,  -1,  

## CHECK FOR POOLING of c1 DIFFERENCE

In [68]:
c1_pool = out_dict['pool_c1']
print(c1_pool[0,0,:,:].int_repr())
c1_pool_manual = torch.load('img_01.png_pool_c1.pt')
print(c1_pool_manual[0,0,:,:])
print(torch.sum(c1_pool_manual - c1_pool.int_repr()))

tensor([[67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67],
        [67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67],
        [67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67],
        [67, 67, 67, 68, 67, 72, 72, 72, 72, 71, 67, 67, 67, 67],
        [67, 67, 73, 73, 77, 77, 74, 67, 67, 67, 67, 67, 67, 67],
        [67, 71, 73, 72, 62, 66, 69, 69, 69, 70, 67, 67, 67, 67],
        [67, 67, 62, 68, 70, 71, 70, 67, 67, 68, 66, 67, 67, 67],
        [67, 67, 68, 71, 68, 67, 67, 67, 68, 66, 66, 67, 67, 67],
        [67, 67, 67, 67, 67, 67, 67, 67, 67, 65, 67, 67, 67, 67],
        [67, 67, 67, 67, 67, 67, 67, 66, 68, 66, 68, 67, 67, 67],
        [67, 67, 67, 67, 67, 67, 67, 68, 69, 65, 67, 67, 67, 67],
        [67, 67, 67, 67, 67, 67, 67, 69, 67, 67, 68, 67, 67, 67],
        [67, 67, 67, 67, 67, 67, 67, 69, 63, 68, 67, 67, 67, 67],
        [67, 67, 67, 67, 67, 67, 69, 67, 66, 68, 67, 67, 67, 67]],
       dtype=torch.uint8)
tensor([[67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 

In [69]:
relu_c1 = out_dict['relu_c1']
relu_c1_manual = torch.load('img_01.png_relu_c1.pt')
print(torch.sum(relu_c1_manual-relu_c1.int_repr()))

tensor(34876)


In [41]:
import pandas as pd
conv_c2 = out_dict['c2']
print(conv_c2.shape)
print(conv_c2[0,0,:,:].int_repr())
for i in [7,15]:
    per_ch = torch.tensor(pd.read_csv(f'c2_{i}_c.csv', header=None).values)[:,:10]
    print(f'channel_{i}', torch.sum(per_ch.unsqueeze(0).unsqueeze(0) - conv_c2[0,i,:,:].int_repr()))
# conv_c2_manual = torch.load('img_01.png_c2.pt')
# print(torch.sum(conv_c2.int_repr() - conv_c2_manual))

torch.Size([1, 16, 10, 10])
tensor([[83, 83, 83, 82, 82, 80, 78, 77, 78, 79],
        [83, 83, 82, 82, 80, 79, 79, 78, 77, 78],
        [83, 82, 81, 81, 83, 85, 81, 79, 82, 84],
        [82, 80, 80, 80, 82, 79, 78, 80, 81, 81],
        [83, 83, 84, 81, 78, 80, 80, 76, 75, 78],
        [80, 79, 76, 72, 77, 81, 78, 71, 74, 80],
        [81, 78, 74, 76, 79, 81, 79, 76, 81, 85],
        [83, 80, 78, 78, 80, 81, 80, 81, 85, 84],
        [82, 82, 82, 82, 82, 81, 81, 83, 84, 83],
        [79, 79, 79, 78, 77, 78, 80, 82, 82, 82]], dtype=torch.uint8)
channel_7 tensor(0., dtype=torch.float64)
channel_15 tensor(0., dtype=torch.float64)


In [46]:
c2_pool_relu = out_dict['pool_c2']
for i in [2,3]:
    per_ch = torch.tensor(pd.read_csv(f'c2pr_{i}_c.csv', header=None).values)[:,:5]
    print(f'channel_{i}', torch.sum(per_ch.unsqueeze(0).unsqueeze(0) - c2_pool_relu[0,i,:,:].int_repr()))
# print(c2_pool_relu.int_repr())
# c2_pool_relu_manual = torch.load('img_01.png_relu_c2.pt')
# print(torch.sum(c2_pool_relu_manual - c2_pool_relu.int_repr()))

channel_2 tensor(0., dtype=torch.float64)
channel_3 tensor(0., dtype=torch.float64)


In [72]:
c3_conv_relu = out_dict['c3']
c3_conv_relu_manual = torch.load('img_01.png_relu_c3.pt')
print(c3_conv_relu_manual)
print(torch.sum(c3_conv_relu_manual - c3_conv_relu))

tensor([[74, 74, 74, 74, 74, 80, 74, 80, 74, 74, 74, 74, 80, 74, 74, 74, 74, 74,
         74, 78, 74, 74, 74, 74, 74, 74, 74, 74, 74, 74, 74, 74, 74, 74, 74, 74,
         74, 74, 74, 74, 74, 74, 74, 74, 74, 74, 74, 74, 74, 83, 74, 74, 74, 78,
         74, 74, 77, 74, 74, 74, 78, 74, 81, 74, 74, 74, 74, 74, 74, 74, 74, 74,
         74, 74, 74, 74, 74, 74, 74, 74, 74, 74, 74, 74, 74, 74, 74, 74, 74, 74,
         74, 74, 74, 74, 78, 74, 74, 74, 74, 74, 74, 74, 74, 75, 74, 74, 74, 74,
         74, 74, 74, 74, 74, 74, 77, 74, 74, 74, 77, 78]], dtype=torch.uint8)
tensor(4862)


In [73]:
fc1_result = out_dict['fc1']
# print(fc1_result.int_repr())
fc1_relu = out_dict['fc1_relu']
print(fc1_relu.int_repr())
fc1_relu_manual = torch.load('img_01.png_fc1_relu.pt')
print(fc1_relu_manual - fc1_relu.int_repr())

tensor([[67, 67, 67, 70, 68, 67, 70, 67, 69, 67, 67, 67, 67, 67, 67, 67, 67, 67,
         67, 68, 69, 70, 67, 73, 72, 67, 71, 67, 73, 71, 67, 75, 67, 67, 67, 67,
         67, 67, 67, 67, 71, 74, 67, 76, 67, 67, 67, 71, 74, 67, 67, 69, 67, 67,
         67, 68, 67, 72, 67, 67, 67, 67, 67, 67, 67, 69, 74, 68, 67, 67, 70, 67,
         67, 67, 67, 73, 67, 67, 67, 69, 67, 67, 67, 67]], dtype=torch.uint8)
tensor([[  2,   3,   0,  11, 255,   0, 253,   0, 254,   0,   0,   0,   0,   6,
           0,   7,   0,   0,   8,   5,   8, 255,   0, 250, 251,   0, 252,   0,
         250, 252,  16, 248,   0,  13,   0,   0,   0,   0,  14,   0, 252, 250,
           0, 247,   8,  13,   0, 252, 249,   0,   0, 254,   8,   4,  11, 255,
           0, 251,   0,   0,   0,  11,   0,   7,   0, 254, 249, 255,  12,   8,
           5,   0,   0,   5,   0, 254,   0,  18,   0, 254,   0,   7,   0,   0]],
       dtype=torch.uint8)


In [74]:

# from torchvision.utils import save_image
# torch.manual_seed(42)  # setting random seed

# # Create output folders
# output_img_dir = Path("results/images")
# output_img_dir.mkdir(parents=True, exist_ok=True)
# output_txt_path = Path("results/results.txt")

# # Clear or create the results.txt file
# with open(output_txt_path, 'w') as f:
#     f.write("idx, ground_truth, predicted\n")

# rows, cols = 2, 6
# fig = plt.figure(figsize=(12, 4))
# for i in range(1, (rows * cols) + 1):
#     random_idx = torch.randint(0, len(test_dataset), size=[1]).item()
#     img, label_gt = test_dataset[random_idx]
#     print(random_idx, img.shape, label_gt)
#     # print(img)
#     img_tensor = img.unsqueeze(dim=0).to(device)
#     # torch.save(img_tensor, f'img_{i:02d}.pt')  # Save the tensor for later use
#     # get the stuff with the highest confidence(?)
#     # label_pred = torch.argmax(net(img_tensor)).item()
#     with torch.no_grad():
#         print(net.quant(img_tensor).int_repr())

#         output, out_dict = net(img_tensor)
#         prob = torch.softmax(output, dim=1)[0]  # shape: [10]
#         label_pred = torch.argmax(prob).item()
#     # === Save image to PNG ===
#     img_filename = f"img_{i:02d}.png"
#     save_image(img, output_img_dir / img_filename)

#     # === Write result to text file ===
#     with open(output_txt_path, 'a') as f:
#         # f.write(f"{img_filename}, {class_names[label_gt]}, {class_names[label_pred]}\n")
#         prob_str = ", ".join([f"{class_names[i]}: {prob[i]:.4f}" for i in range(10)])
#         f.write(f"{img_filename}, GT: {class_names[label_gt]}, Pred: {class_names[label_pred]} → [{prob_str}]\n")    # === Plot image ===
#     fig.add_subplot(rows, cols, i)
#     img_display = img.permute(1, 2, 0)  # CxHxW → HxWxC
#     plt.imshow(img_display, cmap='gray')
#     color = 'g' if label_pred == label_gt else 'r'
#     plt.title(class_names[label_pred], color=color)
#     plt.axis('off')
#     break
# # print(out_dict)
# # torch.save(out_dict['c1'], f'img_01.png_c1_pytorch.pt')

# plt.tight_layout()
# plt.show()

In [75]:
fc2_result = out_dict['fc2']
print(fc2_result.int_repr())
net

tensor([[77, 80, 80, 82, 80, 81, 73, 90, 79, 83]], dtype=torch.uint8)


LeNet5_Dict_Out(
  (quant): Quantize(scale=tensor([0.0256]), zero_point=tensor([17]), dtype=torch.quint8)
  (dequant): DeQuantize()
  (c1): QuantizedConv2d(1, 6, kernel_size=(5, 5), stride=(1, 1), scale=0.13094060122966766, zero_point=68, padding=(2, 2))
  (c2): QuantizedConv2d(6, 16, kernel_size=(5, 5), stride=(1, 1), scale=0.33193832635879517, zero_point=83)
  (c3): QuantizedConv2d(16, 120, kernel_size=(5, 5), stride=(1, 1), scale=0.5925276279449463, zero_point=74)
  (relu): ReLU()
  (max_pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): QuantizedLinear(in_features=120, out_features=84, scale=0.5568221807479858, zero_point=67, qscheme=torch.per_tensor_affine)
  (fc2): QuantizedLinear(in_features=84, out_features=10, scale=1.0586904287338257, zero_point=84, qscheme=torch.per_tensor_affine)
)